In [1]:
import os
from dotenv import load_dotenv

from pathlib import Path
import sys

PATH_PROJECT_ROOT_DIR = Path(os.getcwd()).resolve().parent
PATH_SRC_DIR = os.path.join(PATH_PROJECT_ROOT_DIR, "src/")

sys.path.append(str(PATH_SRC_DIR))

In [2]:
from pipeline_scripts.api_twitter_helper import fetch_recent_tweets
from pipeline_scripts.nlp_model_helper import sentiment_analysis


In [3]:
# Safe mode, you can write your token in a .env or specify it directly into the function.

## X API Bearer Token
BEARER_TOKEN = os.getenv("BEARER_TOKEN") 

## Hugging Face Token
TOKEN_HF = os.getenv("TOKEN_HF") 

## Broker (Alpaca) API credentials
BROKER_TOKEN = os.getenv("BROKER_API_KEY") 
BROKER_SECRET = os.getenv("BROKER_API_SECRET")


MODEL_NAME = "StephanAkkerman/FinTwitBERT-sentiment" 
TICKER = "ETH/USD"
ASSET_NAME =  "Ethereum" # If you do not want to search tweets for company ticker name you can specify it here

In [4]:
from pipeline_scripts.api_twitter_helper import fetch_recent_tweets

input_data = fetch_recent_tweets(bearer_token=BEARER_TOKEN,
                                 accounts=None,
                                 assets=ASSET_NAME if ASSET_NAME else TICKER,
                                 max_results=10)

SSLError: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/tweets/search/recent?max_results=10&tweet.fields=id%2Ctext%2Ccreated_at%2Cauthor_id&sort_order=recency&query=%28E+OR+t+OR+h+OR+e+OR+r+OR+e+OR+u+OR+m%29+-is%3Aretweet+-is%3Areply (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1016)')))

In [ ]:
tweets = [
    "Bitcoin just broke past $70,000! Bulls are celebrating everywhere.",
    "Another dip in Bitcoin price today. Is the rally over?",
    "Crypto market is volatile, but Bitcoin seems to be holding strong.",
    "Bought Bitcoin at $55,000 last month. Now it's $68,000. Feeling good!",
    "My Bitcoin portfolio is down 10%. Should I hold or sell?",
    "Regulators are talking about new Bitcoin rules. Uncertainty ahead.",
    "Finally reached my goal of owning 1 full Bitcoin!",
    "Bitcoin mining difficulty just hit a new high. Network is stronger than ever.",
    "Inflation worries are driving more investors to Bitcoin.",
    "Lost some money trading Bitcoin this week. Tough lessons learned.",
    "Why is everyone bullish on Bitcoin again? Is it sustainable?",
    "Received my first Bitcoin payment for freelance work. Crypto is the future!",
    "Bitcoin is up as traditional markets show weakness. Digital gold in action.",
    "Missed the last Bitcoin pump. Hoping for another entry soon.",
    "Bitcoin transaction fees dropped today. Good news for users.",
    "Got approved to buy Bitcoin on my favorite exchange. Excited to start!",
    "Layoffs in crypto companies, but Bitcoin development continues.",
    "Finally paid off my credit card using profits from Bitcoin trading.",
    "Not sure what to make of Bitcoin's price swings. Staying cautious.",
    "Major exchange outage during Bitcoin rally. Missed my trade!"
]

res = sentiment_analysis(token_hf=TOKEN_HF, model_to_use=MODEL_NAME, data=tweets)
# res2 = sentiment_analysis(token_hf=TOKEN_HF, model_to_use=MODEL_NAME, data=tw_cleaned)

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [35]:
from pipeline_scripts.trading_signal import generate_trading_signal
ACTION = generate_trading_signal(df=res, buy_threshold=0.3, sell_threshold=-0.3)
type(ACTION)


NameError: name 'res' is not defined

In [9]:
os.environ["REQUESTS_CA_BUNDLE"] = "/etc/pki/tls/certs/sanofi-ca.crt" ################# DELETE ###############


In [34]:
from pipeline_scripts.trader_helper import AlpacaTrader
trader = AlpacaTrader(
    api_key=BROKER_TOKEN,
    secret_key=BROKER_SECRET)
print("Account ID: ", trader.trading_client.get_account().account_number, "\nPortfolio value: ",trader.trading_client.get_account().portfolio_value,"$") 

# trader.trading_client.cancel_orders()



Account ID:  PA3IVEW0F5GS 
Portfolio value:  99858.95 $


In [ ]:
trader.execute_action(
    symbol="AAPL",
    asset_type="stock",
    action=ACTION[1],
    qty= 0.2,
    threshold_pct=5.0,
    timeframe_unit="Hour",
    limit=10,
)

trader.trading_client.get_orders()

[]

In [33]:
trader.trading_client.get_clock()

{   'is_open': False,
    'next_close': datetime.datetime(2025, 8, 4, 16, 0, tzinfo=TzInfo(-04:00)),
    'next_open': datetime.datetime(2025, 8, 4, 9, 30, tzinfo=TzInfo(-04:00)),
    'timestamp': datetime.datetime(2025, 8, 3, 11, 45, 37, 739805, tzinfo=TzInfo(-04:00))}

In [32]:
trader.trading_client.get_clock()['is_open']  # Check if the market is open

TypeError: 'Clock' object is not subscriptable